# Model integration

This tutorial demonstrate how an openalea model can be exported as an IPM-compliant web service

## Python Model creation

Let export the folowing simple model, that return some risk level, as a function of air temperature and a user-defined temperature threshold:

In [1]:
def t_risk(tair, threshold=15):
    if tair <= threshold:
        return 0
    else:
        return 1

In [2]:
t_risk(0), t_risk(20), t_risk(20, threshold=20)

(0, 1, 0)

## Import in OpenAlea

Let import it in OpenAlea, by embending it in a OpenAlea Node:

In [3]:
from openalea.core.node import FuncNode
from openalea.core import IFloat, IInt

In [4]:
inputs = (dict(name='tair', interface=IFloat, value=None),
          dict(name='threshold', interface=IFloat, value=15))
outputs = (dict(name='Risk', interface=IInt), )
my_node = FuncNode(inputs, outputs, t_risk)
my_node.name='TRISK'

In [5]:
my_node((0,)), my_node((20,)),my_node((20,20))

(0, 1, 0)

In [6]:
my_node.name, my_node.input_desc, my_node.output_desc

('TRISK',
 [{'name': 'tair', 'interface': IFloat, 'value': None},
  {'name': 'threshold', 'interface': IFloat, 'value': 15}],
 [{'name': 'Risk', 'interface': IInt}])

## Create IPM model

In [7]:
from openalea.dss.dss_factory import dss_factory
help(dss_factory)

Help on function dss_factory in module openalea.dss.dss_factory:

dss_factory(model_id, node, factory=None, interval=86400, weather_parameters=None, config_params=None, decision_support=None, meta=None)
    Transform an openalea node in a IPM model json descriptor and generate a fastAPI script to launch webservice
    
    Args:
        node: the node to be exported
        factory
    
        : the node factory associated to the node (optional)
        interval: the time step of the model (s)
        weather_parameters: a mapping between node input name and weather data codes, if any.
        None if none of the input is a weather data
        parameters: a list of node input name to be exposed as config parameters in IPM-Decison platform
        meta : model meta informations
    Returns:
        model: a json-like dict describing the model
        dss_service: a string containing the script to be run for launching the web service



We will define a simple decision_support function that help a user interpret the risk level of our model

In [8]:
decision_support = list(range(2))
decision_support[0] = {'explanation': 'Risk is low',
                      'recommended_action': 'No particular action is required'}
decision_support[1] = {'explanation': 'Risk is high',
                      'recommended_action': 'Be carreful !'}

In [9]:
input_mapping = {'weather_parameters': {'tair': 1002}, 'config_params': ['threshold']}

ipm_model, service = dss_factory(my_node.name, my_node, decision_support=decision_support, **input_mapping)

In [10]:
ipm_model

{'id': 'TRISK',
 'name': 'Sample model',
 'version': '0.0.0',
 'purpose': 'Sample meta informations generated by openalea dss facctory',
 'description': 'Please provide here a concise description of the model',
 'type_of_decision': 'Short-term tactical',
 'type_of_output': 'Risk indication',
 'description_url': None,
 'citation': None,
 'keywords': None,
 'platform_validated': False,
 'pests': [],
 'crops': [],
 'authors': [{'name': 'Anonymous',
   'email': 'anonymous@mail',
   'organization': 'Unknown'}],
 'valid_spatial': {'countries': [], 'geoJSON': '{}'},
 'execution': {'type': 'ONTHEFLY',
  'endpoint': 'http://127.0.0.1:8000/TRISK/',
  'form_method': 'post',
  'content_type': 'application/json',
  'input_schema_categories': {'hidden': ['modelId'],
   'system': [],
   'user_init': [],
   'triggered': [],
   'internal': []},
  'input_schema': {'type': 'object',
   'properties': {'configParameters': {'title': 'Configuration parameters',
     'type': 'object',
     'properties': {'tim

## Launch the Webservice

In [11]:
from openalea.dss.dss_factory import start_service, stop_service

http, handler = start_service(service)

In [12]:
from IPython.display import IFrame

In [13]:
IFrame(http + '/docs', 800,400)

## Try it out using the fake input generated below !

In [14]:
import agroservices.ipm.fakers as ipm_fakers
import json

In [15]:
input_data = ipm_fakers.input_data(ipm_model)
json.dumps(input_data)

'{"configParameters": {"timeZone": "Europe/Oslo", "timeStart": "2023-03-01", "timeEnd": "2023-09-01", "threshold": 15.0}, "modelId": "TRISK", "weatherData": {"weatherParameters": [1002], "timeStart": "2023-03-01T00:00:00+01:00", "timeEnd": "2023-09-01T00:00:00+02:00", "interval": 86400, "locationWeatherData": [{"longitude": 71.97115723766738, "latitude": 50.69160831176915, "altitude": 0, "data": [[1.3], [1.6], [2.9], [7.3], [9.9], [8.7], [0.5], [1.0], [0.6], [7.0], [3.6], [8.6], [4.3], [3.9], [7.9], [1.4], [0.8], [4.9], [1.9], [1.5], [5.3], [6.2], [1.7], [6.6], [1.1], [9.5], [9.9], [1.4], [4.7], [6.0], [4.4], [7.2], [3.8], [2.8], [5.4], [4.8], [4.2], [8.4], [2.6], [4.2], [6.8], [9.8], [1.6], [6.5], [0.3], [2.2], [0.4], [1.6], [7.2], [3.2], [8.3], [9.1], [4.7], [7.4], [3.5], [6.0], [0.6], [0.5], [5.3], [3.3], [0.4], [7.8], [1.8], [9.7], [8.0], [3.0], [8.8], [7.2], [4.8], [8.5], [7.1], [1.3], [5.1], [3.0], [1.2], [3.8], [8.3], [9.5], [9.9], [2.0], [6.4], [6.4], [8.4], [8.3], [4.5], [2.0]

## Test the exported model

In [16]:
from agroservices.ipm.ipm import IPM

In [17]:
ipm = IPM()
input_data = ipm_fakers.input_data(ipm_model)
input_data

{'configParameters': {'timeZone': 'Europe/Oslo',
  'timeStart': '2023-03-01',
  'timeEnd': '2023-09-01',
  'threshold': 15.0},
 'modelId': 'TRISK',
 'weatherData': {'weatherParameters': [1002],
  'timeStart': '2023-03-01T00:00:00+01:00',
  'timeEnd': '2023-09-01T00:00:00+02:00',
  'interval': 86400,
  'locationWeatherData': [{'longitude': 0.7793325221386915,
    'latitude': 61.65501058067173,
    'altitude': 0,
    'data': [[6.5],
     [4.5],
     [7.6],
     [2.2],
     [7.1],
     [6.9],
     [8.7],
     [1.6],
     [3.7],
     [8.9],
     [6.0],
     [6.1],
     [5.7],
     [3.6],
     [3.9],
     [3.9],
     [3.0],
     [3.5],
     [4.7],
     [8.8],
     [5.2],
     [8.8],
     [1.7],
     [3.5],
     [1.7],
     [7.0],
     [5.3],
     [5.8],
     [2.2],
     [0.3],
     [8.4],
     [7.4],
     [7.1],
     [1.7],
     [4.1],
     [8.1],
     [4.4],
     [2.6],
     [5.1],
     [6.8],
     [6.4],
     [3.8],
     [4.1],
     [8.5],
     [3.3],
     [9.9],
     [7.1],
     [6.1],
 

In [20]:
output=ipm.run_model(ipm_model, input_data)
output

{'timeStart': '2023-03-01T00:00:00+01:00',
 'timeEnd': '2023-09-01T00:00:00+02:00',
 'interval': 86400,
 'resultParameters': ['Risk'],
 'locationResult': [{'longitude': 0.7793325221386915,
   'latitude': 61.65501058067173,
   'altitude': 0.0,
   'data': [[0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
    [0],
  

In [21]:
import pandas
df = pandas.DataFrame(dict(zip(output['resultParameters'],zip(*output['locationResult'][0]['data']))))
df

,Risk
0,0
1,0
2,0
3,0
4,0
...,...
179,0
180,0
181,0
182,0


# Stop service

In [20]:
stop_service(handler)

NoSuchProcess: process PID not found (pid=14536)

In [18]:
IFrame(http + '/docs', 800,400)